In [1]:
#Installing required packages

!pip install pulp
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
#Loading excel file into python
import pandas as pd

# Load the Excel file to check its structure
file_path = 'toy-stores 1.xlsx'
data = pd.ExcelFile(file_path)

# Display the sheet names and inspect the content of the first sheet
data.sheet_names

['customer', 'candidate']

In [3]:
# Load and display the content of each sheet to understand the structure
customer_data = data.parse('customer')
candidate_data = data.parse('candidate')

customer_data.head(), candidate_data.head()

(       City        Lon       Lat  Demand (pallets)
 0  Amarillo  101.81880  35.20350              5893
 1    Austin   97.75066  30.30493             18502
 2    Dallas   96.76747  32.79315             36936
 3   Houston   95.38733  29.76805             63937
 4    Irving   96.96934  32.85769              6235,
        City        Lon       Lat  Fixed Cost  Capacity
 0  Amarillo  101.81880  35.20350    52074720     40000
 1    Austin   97.75066  30.30493    34443780     50000
 2   El Paso  106.43750  31.84925    41619720     33000
 3   Houston   95.38733  29.76805     7411680    100000
 4    Laredo   99.48660  27.53430    25876140     27000)

In [4]:
from geopy.distance import geodesic

# Extract relevant data
customers = customer_data[['City', 'Lon', 'Lat', 'Demand (pallets)']]
candidates = candidate_data[['City', 'Lon', 'Lat', 'Fixed Cost', 'Capacity']]

# Initialize a distance matrix
distance_matrix = pd.DataFrame(
    index=customers['City'], 
    columns=candidates['City']
)

# Compute great-circle distances between each customer and candidate DC
for customer in customers.itertuples():
    for candidate in candidates.itertuples():
        customer_coords = (customer.Lat, customer.Lon)
        candidate_coords = (candidate.Lat, candidate.Lon)
        distance_matrix.at[customer.City, candidate.City] = geodesic(customer_coords, candidate_coords).miles

# Convert distances to numeric
distance_matrix = distance_matrix.apply(pd.to_numeric)
distance_matrix.head()

City,Amarillo,Austin,El Paso,Houston,Laredo
City,,,,,
Amarillo,0.000000,412.285735,352.784162,530.184914,546.007972
Austin,412.285735,0.000000,525.804947,146.383410,217.872962
Dallas,334.126698,180.976470,569.311538,223.831936,397.076598
Houston,530.184914,146.383410,672.187986,0.000000,292.668595
Irving,321.782109,181.813362,557.930198,232.509308,396.377021


In [5]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Create a list of customer and candidate DC names
customer_names = customers['City'].tolist()
candidate_names = candidates['City'].tolist()

# Extract demands, fixed costs, and capacities
demands = dict(zip(customers['City'], customers['Demand (pallets)']))
fixed_costs = dict(zip(candidates['City'], candidates['Fixed Cost']))
capacities = dict(zip(candidates['City'], candidates['Capacity']))

# Initialize the optimization problem
problem = LpProblem("CFLP", LpMinimize)

# Decision variables
# x[i][j]: 1 if customer i is assigned to DC j, 0 otherwise
x = LpVariable.dicts("Assign", [(i, j) for i in customer_names for j in candidate_names], cat='Binary')

# y[j]: 1 if DC j is opened, 0 otherwise
y = LpVariable.dicts("Open", candidate_names, cat='Binary')

# Objective: Minimize total cost (fixed + transportation)
problem += lpSum(
    fixed_costs[j] * y[j] + lpSum(demands[i] * distance_matrix.loc[i, j] * x[(i, j)] for i in customer_names) 
    for j in candidate_names
)

# Constraints
# Each customer is assigned to exactly one DC
for i in customer_names:
    problem += lpSum(x[(i, j)] for j in candidate_names) == 1

# A DC's capacity must not be exceeded
for j in candidate_names:
    problem += lpSum(demands[i] * x[(i, j)] for i in customer_names) <= capacities[j] * y[j]

# Solve the problem
problem.solve()

# Extract the results
assignments = {i: [j for j in candidate_names if x[(i, j)].varValue > 0.5] for i in customer_names}
opened_dcs = [j for j in candidate_names if y[j].varValue > 0.5]
total_cost = problem.objective.value()

assignments, opened_dcs, total_cost

({'Amarillo': ['Austin'],
  'Austin': ['Laredo'],
  'Dallas': ['Austin'],
  'Houston': ['Houston'],
  'Irving': ['Austin'],
  'Laredo': ['Laredo'],
  'Pasadena': ['Laredo'],
  'San Antonio': ['Houston']},
 ['Austin', 'Houston', 'Laredo'],
 89392207.80855736)

In [ ]:
# Solution Summary:
# Opened Distribution Centers (DCs):

# Austin
# Houston
# Laredo

# Customer Assignments:

# Amarillo → Assigned to Austin
# Austin → Assigned to Laredo
# Dallas → Assigned to Austin
# Houston → Assigned to Houston
# Irving → Assigned to Austin
# Laredo → Assigned to Laredo
# Pasadena → Assigned to Laredo
# San Antonio → Assigned to Houston

# Total Annual Cost: $89,392,207.81